### login.py

In [17]:
def login():
    key="webvitals_config.properties"

    try:
        f = open(key)
        configs = Properties()

        with open(key, 'rb') as config_file:
            configs.load(config_file)

        items_view = configs.items()
        list_keys = []
        print("'" + key + "'" + " found!")    
        for item in items_view:
            list_keys.append(item[0])    
        # Login attempt (1)
        driver.find_element_by_name("ssousername").send_keys(configs.get("USERNAME")[0])
        driver.find_element_by_name("password").send_keys(configs.get("PASSWORD")[0])
        xpath="/html/body/center/table/tbody/tr[3]/td/font/input[1]"
        driver.find_element_by_xpath(xpath).click()
        # Enter Webvitals
        driver.find_element_by_xpath("/html/body/ul/li[1]/a").click()
        
        # Handle errors logging in
        WebDriverWait(driver=driver, timeout=10).until(
        lambda x: x.execute_script("return document.readyState === 'complete'"))
        error_message = "Incorrect username or password."
        errors = driver.find_elements_by_class_name("flash-error")

        # Print errors optionally
        if any(error_message in e.text for e in errors):
            print("[!] Login failed!")
        else:
            print("[+] Login successful!")
            
        f.close()
        
    except IOError:
        print('Enter correct username and password combo to continue')
        count=0
        while count < 3:
            username = getpass.getpass('Enter Username:')
            password = getpass.getpass('Enter Password:')
            try:
                driver.find_element_by_name("ssousername").send_keys(username)
                driver.find_element_by_name("password").send_keys(password)
                xpath="/html/body/center/table/tbody/tr[3]/td/font/input[1]"
                driver.find_element_by_xpath(xpath).click()
                # Enter Webvitals
                driver.find_element_by_xpath("/html/body/ul/li[1]/a").click()
                print('Access granted')
                break
            except NoSuchElementException:
                print('Access denied. Try again.')
                count += 1

### animal_location.py

In [149]:
# Location

def animal_location(ids):
    
    data = pd.DataFrame()
    
    for i, x in enumerate( tqdm(ids) ):
        for i in ids:

            # Go to relocation history in Webvitals
            driver.find_element_by_name("query_input").send_keys(i)
            driver.find_element_by_name("submit").click()

            # Return to Animal Summary
            xpath="/html/body/table[1]/tbody/tr[3]/td/center/table[2]/tbody/tr/td[1]/a"
            driver.find_element_by_xpath(xpath).click()

            # Look for cause of death if animal has passed 
            xpath="/html/body/table[2]/tbody/tr/td[1]/table[2]/tbody/tr[11]"
            cause_of_death = driver.find_element_by_xpath(xpath).text

            # Go to relocation history in Webvitals
            xpath="/html/body/table[1]/tbody/tr[3]/td/center/table[3]/tbody/tr/td[5]/a"
            driver.find_element_by_xpath(xpath).click()

            # Extract html table
            xpath="/html/body/table[2]/tbody/tr/td[1]/center[1]/table"
            tableelement= (
                WebDriverWait(driver,10)

                .until(EC.visibility_of_element_located((By.XPATH, xpath)))
                .get_attribute('outerHTML')
            )        
            table = pd.read_html(str(tableelement))[0]

            # Add column to specify MMU number
            table['MMU'] = i
            first_column = table.pop('MMU')
            table.insert(0, 'MMU', first_column)

            # Add cause of death if animal has passed 
            table['Cause_of_death']=""
            table.loc[table['MMU'] == i, ['Cause_of_death']] = cause_of_death

            # Append dataframes into one dataframe
            data = data.append(table, ignore_index=True)

        # Clean output table   
        data = data.rename(columns={"Time at Locationyr : mon : day" : "Time"})
        data = data.join(data['Time'].str.split(':', 3, expand=True))
        data = data.rename(columns={0:"Year",1:"Month",2:"Day"})
        
        # Subset date of death if animal is dead
        death = (
            data
            .loc[data['Location'] == 'DEAD']
            .rename(columns={"Date In":"Dead"})
        )[['MMU', 'Dead']]
        # Merge dataframes
        df = (
            data
            .merge(death, on='MMU', how='left')
            .dropna()
        )
        # Convert age to numeric
        df['Year'] = pd.to_numeric(df['Year'])
        df['Month'] = pd.to_numeric(df['Month'])
        df['Day'] = pd.to_numeric(df['Day'])
        # List age in months
        day_year = 365.2425
        day_month = 30.436875
        df['Days'] = df.Year.mul(day_year) + df.Month.mul(day_month) + df.Day
        df['Months'] = df.Days.div(day_month)
        df.drop(['Year', 'Month', 'Day', 'Days', 'Time'],inplace=True, axis=1)
        df = df.round(2)
        # Add object to namespace
        globals()['df'] = df
        # Export table
        os.makedirs('data', exist_ok=True)
        output='webvitals_query.csv'
        timestr = time.strftime("data/%Y%m%d-%H%M%S")
        df.to_csv(timestr + '-' + 'relocation_' + output)
        print(df)

### diarrhea_observations.py

In [183]:
# Diarrhea observations

def diarrhea_observations(ids):
    data = pd.DataFrame()
    
    for i in ids:
        # Go to Diarrhea section in Webvitals
        driver.find_element_by_name("query_input").send_keys(i)
        driver.find_element_by_name("submit").click()
        xpath="/html/body/table[1]/tbody/tr[3]/td/center/table[2]/tbody/tr/td[6]/a"
        driver.find_element_by_xpath(xpath).click()

        # Extract html table
        xpath="/html/body/table[2]/tbody/tr/td[1]/center[1]/table"
        tableelement= (
            WebDriverWait(driver,10)
            .until(EC.visibility_of_element_located((By.XPATH, xpath)))
            .get_attribute('outerHTML')
        )        
        table = pd.read_html(str(tableelement))[0]

        # Add column to specify MMU number
        table['MMU'] = i
        first_column = table.pop('MMU')
        table.insert(0, 'MMU', first_column)

        # Append dataframes into one dataframe
        data = data.append(table, ignore_index=True)
        
        # Replace blank cells with 0
        df = data.fillna(0)
        # Replace all diarrhea obs with 1
        df = df.replace(dict.fromkeys(['D', '+D', '~D', '-D'], ['1']))
        # Replace all move obs with 0
        df = df.replace(dict.fromkeys(['M', '+M', '~M', '-M', '+', '-', '~'], ['0']))
        # Sum monthly diarrhea obs
        df['Month_diarrhea_obs'] = df.iloc[:,3:31].astype(int).sum(axis=1, numeric_only=True)
        # Calcualte the # of obs in a 6-month window
        df['Rolling_sum_obs'] = df.Month_diarrhea_obs.rolling(6).sum().fillna(0)
        # Determine if animal meeting qual. for chronic issue
        df['Chronic_diarrhea'] = np.where(
            df['Rolling_sum_obs'] > 45, 1, np.where(
            df['Rolling_sum_obs'] < 45, 0, 0))       
    # Add object to namespace
    globals()['df'] = df
    # Export table
    os.makedirs('data', exist_ok=True)
    output='webvitals_query.csv'
    timestr = time.strftime("data/%Y%m%d-%H%M%S")
    df.to_csv(timestr + '-' + 'diarrhea_obs_' + output)
    print(df)

### weight.py

In [ ]:
# Weight

def weight(ids):
    data = pd.DataFrame()
    
    for i in ids:
        # Go to weight TB in Webvitals
        driver.find_element_by_name("query_input").send_keys(i)
        driver.find_element_by_name("submit").click()
        
        # Return to Animal Summary
        xpath="/html/body/table[1]/tbody/tr[3]/td/center/table[2]/tbody/tr/td[1]/a"
        driver.find_element_by_xpath(xpath).click()
        
        # Extract birthday
        xpath="/html/body/table[2]/tbody/tr/td[1]/table[2]/tbody/tr[5]/td[2]"
        birthday = driver.find_element_by_xpath(xpath).text
        
        xpath="/html/body/table[1]/tbody/tr[3]/td/center/table[3]/tbody/tr/td[10]/a"
        driver.find_element_by_xpath(xpath).click()

        # Extract html table
        xpath="/html/body/table[2]/tbody/tr/td[1]/center[1]/table"
        tableelement= (
            WebDriverWait(driver,10)
            .until(EC.visibility_of_element_located((By.XPATH, xpath)))
            .get_attribute('outerHTML')
        )        
        table = pd.read_html(str(tableelement))[0]

        # Add column to specify MMU number
        table['MMU'] = i
        first_column = table.pop('MMU')
        table.insert(0, 'MMU', first_column)
        
        # Clean up birthday string
        birthday = str(dparser.parse(birthday,fuzzy=True)).split(" ")[0]
        birthday = (
            datetime.datetime.strptime(birthday,'%Y-%m-%d')
                    .strftime('%m-%d-%Y')
                    .split(" ")[0]
        )
        
        # Add birthday column
        table['Birth']=""
        ninth_column = table.pop('Birth')
        table.insert(8, 'Birth', ninth_column)
        table.loc[table['MMU'] == i, ['Birth']] = birthday

        # Append dataframes into one dataframe
        data = data.append(table, ignore_index=True)
    
    # Drop non-informative data
    df = data.drop(columns = ['Body Condition', 'Conception ID',
                              'Days Pregnant', 'TB', 'Tattoo',
                              'Test 1', 'Test 2'])
    # Convert to datetime
    ## List age in months
    day_month = 30.436875
    df[["Birth", "Weighing Date"]] = df[["Birth", "Weighing Date"]].apply(pd.to_datetime)
    
    # Calculate age in days
    df['Age_days'] = (df['Weighing Date'] - df['Birth']).dt.days
    
    # Calculate age in months
    df['Age_months'] = df.Age_days.div(day_month).round(2)
    
    # Reorder columns
    column_names = ['MMU', 'Location', 'Birth',
                    'Weighing Date', 'Age_days',
                    'Age_months', 'Weight']
    df = df.reindex(columns=column_names)
    
    # Add object to namespace
    globals()['df'] = df
    # Export table
    os.makedirs('data', exist_ok=True)
    output='webvitals_query.csv'
    timestr = time.strftime("data/%Y%m%d-%H%M%S")
    df.to_csv(timestr + '-' + 'weight' + output)
    print(df)

### webvitals.py

In [180]:
# Example query for testing
query = ['47455','47510' ,'40459']

In [227]:
import selenium
import datetime
import getpass
import time
import os

import numpy as np
import pandas as pd
import dateutil.parser as dparser

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import NoSuchElementException

from jproperties import Properties

from webdriver_manager.firefox import GeckoDriverManager

from bs4 import BeautifulSoup

# Install Geckodriver
driver = webdriver.Firefox(executable_path=GeckoDriverManager(version="v0.20.0").install())

# # Enter options
# options = FirefoxOptions()
# options.add_argument("--headless")
# driver = webdriver.Firefox(options=options)

# Go to login page
driver.get("https://davos.primate.ucdavis.edu/login/")

login()

animal_location(query)
        
diarrhea_observations(query)

weight(query)

# def snomed(ids):
#     data = pd.DataFrame()
    
#     for i in ids:
#         # Go to snomed in Webvitals
#         driver.find_element_by_name("query_input").send_keys('47455')
#         driver.find_element_by_name("submit").click()
        
#         xpath="/html/body/table[1]/tbody/tr[3]/td/center/table[3]/tbody/tr/td[8]/a"
#         driver.find_element_by_xpath(xpath).click()

#         # Extract html table
#         xpath="/html/body/table[2]/tbody/tr/td[1]/html:errors/center[1]/table"
#         tableelement= (
#             WebDriverWait(driver,10)

#             .until(EC.visibility_of_element_located((By.XPATH, xpath)))
#             .get_attribute('outerHTML')
#         )        
#         table = pd.read_html(str(tableelement))[0]

#         # Add column to specify MMU number
#         table['MMU'] = '47455'
#         first_column = table.pop('MMU')
#         table.insert(0, 'MMU', first_column)

#         # Append dataframes into one dataframe
#         data = data.append(table, ignore_index=True)
#         print(data)



print('driver Title:',driver.title)
print('Driver name:',driver.name)
print('Driver URL:',driver.current_url)
driver.quit()

[WDM] - ====== WebDriver manager ======
[WDM] - Driver [C:\Users\Noah Siegel\.wdm\drivers\geckodriver\win64\v0.20.0\geckodriver.exe] found in cache


'webvitals_config.properties' found!
[+] Login successful!
      MMU   Location      Birth Weighing Date  Age_days  Age_months  Weight
0   47455        NaN 2018-05-27    2018-11-30       187        6.14    1.17
1   47455  NW1409-32 2018-05-27    2018-10-31       157        5.16    1.17
2   47455   SW1604-3 2018-05-27    2018-10-01       127        4.17    1.05
3   47455   SW1604-3 2018-05-27    2018-08-31        96        3.15    0.89
4   47455   SW1604-3 2018-05-27    2018-07-31        65        2.14    0.74
5   47455   SW1604-3 2018-05-27    2018-07-02        36        1.18    0.64
6   47455   SW1604-3 2018-05-27    2018-05-31         4        0.13    0.50
7   47510        NaN 2018-06-14    2018-12-18       187        6.14    1.42
8   47510   NW1409-8 2018-06-14    2018-11-19       158        5.19    1.38
9   47510   NW1411-6 2018-06-14    2018-10-18       126        4.14    1.15
10  47510   NW1411-6 2018-06-14    2018-09-18        96        3.15    0.96
11  47510   NW1411-6 2018-06-

In [228]:
df

,MMU,Location,Birth,Weighing Date,Age_days,Age_months,Weight
0,47455,NaN,2018-05-27,2018-11-30,187,6.14,1.17
1,47455,NW1409-32,2018-05-27,2018-10-31,157,5.16,1.17
2,47455,SW1604-3,2018-05-27,2018-10-01,127,4.17,1.05
3,47455,SW1604-3,2018-05-27,2018-08-31,96,3.15,0.89
4,47455,SW1604-3,2018-05-27,2018-07-31,65,2.14,0.74
5,47455,SW1604-3,2018-05-27,2018-07-02,36,1.18,0.64
6,47455,SW1604-3,2018-05-27,2018-05-31,4,0.13,0.50
7,47510,NaN,2018-06-14,2018-12-18,187,6.14,1.42
8,47510,NW1409-8,2018-06-14,2018-11-19,158,5.19,1.38
9,47510,NW1411-6,2018-06-14,2018-10-18,126,4.14,1.15


### Options

In [ ]:
# Relocation
driver.find_element_by_xpath("/html/body/table[1]/tbody/tr[3]/td/center/table[3]/tbody/tr/td[5]/a").click()
# Diarrhea
driver.find_element_by_xpath("/html/body/table[1]/tbody/tr[3]/td/center/table[2]/tbody/tr/td[6]/a").click()
# Snomed
driver.find_element_by_xpath("/html/body/table[1]/tbody/tr[3]/td/center/table[3]/tbody/tr/td[8]/a").click()
# Weight TB
driver.find_element_by_xpath("/html/body/table[1]/tbody/tr[3]/td/center/table[3]/tbody/tr/td[10]/a").click()